In [ ]:
import xmlrpc
from xmlrpc.server import SimpleXMLRPCServer, SimpleXMLRPCRequestHandler
from xmlrpc.client import ServerProxy, Fault
import datetime
import time
from collections import deque


class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

proxy_server = SimpleXMLRPCServer(("127.0.0.1", 8028), requestHandler=RequestHandler, allow_none=True)

main_server = xmlrpc.client.ServerProxy("http://127.0.0.1:8008", allow_none=True)

stats_server = xmlrpc.client.ServerProxy("http://127.0.0.1:8018", allow_none=True)

# Rate limiting: N запросов в T секунд
RATE_LIMIT_N = 5
RATE_LIMIT_T = 5
# Храним timestamps последних запросов
request_times = deque()


def set_rate_limit(n, t):
    global RATE_LIMIT_N, RATE_LIMIT_T
    RATE_LIMIT_N = n
    RATE_LIMIT_T = t
    return True


proxy_server.register_function(set_rate_limit, 'set_rate_limit')


# Прокся функция с логгированием и РЛ
def proxy_function(*args, **kwargs):
    method_name = proxy_function.method_name

    # === Rate Limiting ===
    current_time = time.time()
    while request_times and request_times[0] < current_time - RATE_LIMIT_T:
        request_times.popleft()

    if len(request_times) >= RATE_LIMIT_N:
        # === ЛИМИТ СРАБОТАЛ ===
        wait_start = time.time()
        wait_until = request_times[0] + RATE_LIMIT_T
        wait_time = wait_until - current_time

        # Ждём
        time.sleep(wait_time)

        # === ЛОГИРУЕМ ОЖИДАНИЕ ===
        event_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        try:
            stats_server.add_log('too_many_requests', event_time, round(wait_time, 3))
        except (Fault, ConnectionRefusedError, Exception):
            pass  # Игнорируем, если stats недоступен

        # Обновляем очередь после ожидания
        current_time = time.time()
        while request_times and request_times[0] < current_time - RATE_LIMIT_T:
            request_times.popleft()

    # Добавляем текущий запрос
    request_times.append(time.time())

    # === Выполнение метода ===
    start_time = time.time()
    event_time_str = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    try:
        method = getattr(main_server, method_name)
        result = method(*args, **kwargs)

        end_time = time.time()
        duration = end_time - start_time

        # Лог успешного вызова
        try:
            stats_server.add_log(method_name, event_time_str, round(duration, 3))
        except (Fault, ConnectionRefusedError, Exception):
            pass

        return result
    except Fault as e:
        print(f"Ошибка XML-RPC при вызове метода {method_name}: {e}")
        raise
    except Exception as e:
        print(f"Неизвестная ошибка при вызове метода {method_name}: {e}")
        raise


# Регистрируем все методы
methods = ['ping', 'now', 'type', 'sum', 'pow', 'black_list_check', 'black_list_check_full',
           'send_back_binary', 'color_inversion', 'send_back_binarization',
           'send_back_binarization_with_percent', 'send_back_flip_vertical']

for method in methods:
    def create_proxy_method(method_name):
        def specific_proxy(*args, **kwargs):
            proxy_function.method_name = method_name
            return proxy_function(*args, **kwargs)

        return specific_proxy


    proxy_server.register_function(create_proxy_method(method), method)

proxy_server.register_introspection_functions()

print("Listening PROXY on port 8028...")
proxy_server.serve_forever()

Listening PROXY on port 8028...
Неизвестная ошибка при вызове метода ping: [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение


127.0.0.1 - - [29/Nov/2025 10:20:20] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 10:22:34] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 10:22:34] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 10:22:49] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 10:22:49] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 10:23:04] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 10:23:04] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 10:23:19] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 10:23:19] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 10:23:34] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 10:23:34] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 10:23:49] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 10:23:49] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 10:24:04] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 10:24:04] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/202